In [35]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

In [2]:
# Load dataset
df = pd.read_csv("./data/household_power_consumption_cleaned.csv", parse_dates=['datetime'], index_col='datetime')

- However Contracted, The choosing of the below two variables is the result of extended meditation.

In [18]:
df = df[['Global_active_power', 'Global_reactive_power']]

In [19]:
df.head()

,Global_active_power,Global_reactive_power
datetime,,
2006-12-16 17:24:00,4.216,0.418
2006-12-16 17:25:00,5.360,0.436
2006-12-16 17:26:00,5.374,0.498
2006-12-16 17:27:00,5.388,0.502
2006-12-16 17:28:00,3.666,0.528


In [26]:
# Split into train (70%) and test (30%)
train_size = int(len(df) * 0.7)
train, test = df.iloc[:train_size], df.iloc[train_size:]

In [27]:
def split_sequences(dataset, n_steps = 1):
    x,  y = [], []

    for i in range(len(dataset) - n_steps):
        xs = dataset[i: (i + n_steps)]
        ys = dataset[i + n_steps]

        x.append(xs)
        y.append(ys)

    return np.array(x), np.array(y)

In [38]:
n_steps = 5
X_train, y_train = split_sequences(train.to_numpy(), n_steps=n_steps)
X_test, y_test = split_sequences(test.to_numpy(), n_steps=n_steps)

In [47]:
X_train[:1], y_train[:1]

(array([[[4.216, 0.418],
         [5.36 , 0.436],
         [5.374, 0.498],
         [5.388, 0.502],
         [3.666, 0.528]]]),
 array([[3.52 , 0.522]]))

In [54]:
import tensorflow as tf

In [55]:
n_features = X_train.shape[2]

In [66]:
# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(n_steps, n_features)),
    tf.keras.layers.LSTM(200, activation='relu'),
    tf.keras.layers.Dense(n_features)
])

In [67]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.compile(optimizer='adam', loss='mse')
callbacks = [early_stopping]
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 200)            │       162,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │           402 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 162,802 (635.95 KB)

 Trainable params: 162,802 (635.95 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(X_train, y_train, epochs=10, batch_size=256, validation_split=0.2, callbacks=callbacks, verbose=1)

Epoch 1/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 66s 14ms/step - loss: 0.0719 - val_loss: 0.0289
Epoch 2/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - loss: 0.0395 - val_loss: 0.0286
Epoch 3/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0389 - val_loss: 0.0272
Epoch 4/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0375 - val_loss: 0.0282
Epoch 5/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0363 - val_loss: 0.0262
Epoch 6/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0359 - val_loss: 0.0259
Epoch 7/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0358 - val_loss: 0.0259
Epoch 8/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0350 - val_loss: 0.0260
Epoch 9/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 58s 13ms/step - loss: 0.0350 - val_loss: 0.0260
Epoch 10/10
4540/4540 ━━━━━━━━━━━━━━━━━━━━ 57s 13ms/step - loss: 0.0348 - val_loss: 0.0255


In [79]:
model.predict(X_train[-1])

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [ ]:
# Compare real vs predicted
plt.figure(figsize=(12, 6))
plt.plot(test.index, Y_test.values, label='Actual', color='blue')
plt.plot(test.index, y_pred, label='Predicted', color='red')
plt.legend()
plt.xlabel('Time')
plt.ylabel('Global Active Power')
plt.title('Real vs Predicted')
plt.show()

In [ ]:

# Error Metrics
mae = mean_absolute_error(Y_test, y_pred)
rmse = np.sqrt(mean_squared_error(Y_test, y_pred))
print(f'MAE: {mae}, RMSE: {rmse}')
